<a href="https://colab.research.google.com/github/yingchenjin/IANAL/blob/main/legally_trained_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Mount Drive to access files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Preprocess Data and remove extratext

In [ ]:
!pip install transformers sentence-transformers PyPDF2
!pip install ace_tools
!pip install faiss-cpu
!pip install pymupdf

import pandas as pd
import re
import html
from lxml import etree
import os
import fitz
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer, util
import glob
import torch
from transformers import AutoTokenizer, AutoModel
import PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
import spacy

try:
    nlp = spacy.load("en_core_web_sm")
    print("spaCy model loaded successfully!")
except OSError:
    print("spaCy model not found. Try reinstalling.")

spaCy model loaded successfully!


In [ ]:


# Load spaCy English NLP model
nlp = spacy.load("en_core_web_sm")

# Function to remove HTML tags using lxml
def remove_html(text):
    if pd.isna(text):
        return ""

    # Decode HTML entities (e.g., &lt; becomes <, &amp; becomes &)
    text = html.unescape(text)

    # Parse with lxml to remove tags
    parser = etree.HTMLParser()
    tree = etree.fromstring(f"<root>{text}</root>", parser)
    text = "".join(tree.itertext())

    return text

# Function for advanced text cleaning
def clean_text(text):
    text = remove_html(text)  # First, remove HTML

    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)

    # Remove special characters, numbers, and extra spaces
    text = re.sub(r"[^a-zA-Z\s]", " ", text)

    # Convert to lowercase
    text = text.lower()

    # Process text using spaCy (Tokenization, Stopword Removal, Lemmatization)
    doc = nlp(text)
    words = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

    # Join words back into a single string
    return " ".join(words)

# Load dataset
file_path = '/content/drive/MyDrive/Colab/AES/ossianal_posts.csv'
df = pd.read_csv(file_path)

# Combine question and answer text and apply cleaning
df["cleaned_question"] = (df["question_body"].fillna("")).apply(clean_text)
df["cleaned_answer"] = (df["answer_body"].fillna("")).apply(clean_text)

# Save the processed dataset
processed_file_path = "/content/drive/MyDrive/Colab/AES/processed_merged_ianal.csv"
df.to_csv(processed_file_path, index=False)

  # Save the cleaned dataset locally
print("Processed data saved as processed_merged_ianal.csv")

Processed data saved as processed_merged_ianal.csv


##Retreival Based Algorithm
Python script that demonstrates a retrieval-based approach using Hugging Face Transformers and the Sentence Transformers library. The system scores (ranks) how well each response aligns with your legal guidelines. The idea is to measure the similarity or "alignment" between the content of each forum answer and the legal text.

In [ ]:
# --- Step 1: Extract Text from the Legal PDF ---

def extract_text_from_pdfs(directory):
    pdf_texts = []
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(directory, filename)
            with fitz.open(pdf_path) as doc:
                text = "\n".join([page.get_text() for page in doc])
                pdf_texts.append(text)
    return pdf_texts
"""
def extract_text_from_pdfs(directory):
    pdf_texts = []
    for filename in os.listdir(directory):
      if filename.endswith(".pdf"):
        pdf_path = os.path.join(directory, filename)
        text = ""
        with open(pdf_path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            for page in reader.pages:
                text += page.extract_text() + "\n"
        pdf_texts.append(text)
    return text
"""
pdf_dir = "/content/drive/MyDrive/Colab/AES/licences"
legal_texts = extract_text_from_pdfs(pdf_dir)


# Optionally, split the legal text into chunks (e.g., by paragraphs or sections)
# For this example, we'll split by two newlines which often separate paragraphs.
legal_chunks = []
for legal_text in legal_texts:  # directly iterate over the texts
    legal_chunks += [chunk.strip() for chunk in legal_text.split("\n") if len(chunk.strip()) > 100]
print("Number of legal chunks:", len(legal_chunks))

# --- Step 2: Load Forum Q&A Data ---
# Assuming your CSV has a column named "text" for the forum responses.
forum_df = pd.read_csv("/content/drive/MyDrive/Colab/AES/processed_merged_ianal_trim.csv")
print(forum_df.head())

# --- Step 3: Generate Embeddings using a Legal Transformer Model ---
# We use a legal-domain transformer model (e.g., LegalBERT) to encode texts.
model_checkpoint = "nlpaueb/legal-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModel.from_pretrained(model_checkpoint)

def encode_texts(texts, tokenizer, model, max_length=512):
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=max_length)
        with torch.no_grad():
            outputs = model(**inputs)
        # Use the [CLS] token embedding as a representation
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu()
        embeddings.append(cls_embedding)
    return torch.stack(embeddings)

# Encode legal chunks
print("Encoding legal document chunks...")
legal_embeddings = encode_texts(legal_chunks, tokenizer, model)

# Encode forum responses
forum_texts = forum_df["cleaned_answer"].tolist()
print("Encoding forum responses...")
forum_embeddings = encode_texts(forum_texts, tokenizer, model)

# --- Step 4: Compute Similarity Scores and Rank Legal Chunks ---
def compute_alignment_score(forum_embedding, legal_embeddings):
    # Compute cosine similarities between a forum response and all legal chunks
    cosine_scores = util.cos_sim(forum_embedding, legal_embeddings)
    # Get the maximum similarity score as a proxy for alignment
    max_score = torch.max(cosine_scores).item()
    return max_score

# Compute alignment scores for all forum responses
alignment_scores = []
for embed in forum_embeddings:
    score = compute_alignment_score(embed.unsqueeze(0), legal_embeddings)
    alignment_scores.append(score)

forum_df["alignment_score"] = alignment_scores

# Optionally, label a response as "aligned" if the score exceeds a threshold.
# You may adjust the threshold based on your experiments.
threshold = 0.7
forum_df["predicted_alignment"] = forum_df["alignment_score"].apply(lambda x: "aligned" if x >= threshold else "not aligned")

print(forum_df.head())

# Save the results
forum_df.to_csv("forum_alignment_results.csv", index=False)


Number of legal chunks: 6974
                                    cleaned_question  \
0  publisher   release publication follow license...   
1  title say   basically   gplv   require use gpl...   
2  section      mpl      state \n\n\n   distribut...   
3  s tricky situation didn t encounter  \n\n\n co...   
4  look open source license enlarged scope consid...   

                                      cleaned_answer  
0  \n cc discuss good practice attribution great ...  
1  question title   yes \n philip kendall s answe...  
2  possible viewpoint   ianal   consider \n\n ala...  
3  \n party   contract   one allow code binary   ...  
4  ianal ve rodeo   simple definition software wr...  


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Encoding legal document chunks...


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]